# FaceGuard FaceEmbedding Training

The following training approach utilizes triplet loss to train a faceembedding model 

In [ ]:
import torch 
import os   
from os.path import exists, join
import numpy as np
from PIL import Image  
import matplotlib.pyplot as plt 
import matplotlib.image as mpimg 
from torch.utils.tensorboard import SummaryWriter 
from torchvision import datasets, models, transforms  

## Basic Configuration 
Please set the variable to the downloaded folder set

In [ ]:
var_dataset_folder = 'dataset/lfw/lfw'

#Place where the dataset split copy should be placed
var_dataset_split_folder = 'dataset/lfw/triplet_dataset'

var_val_split = 0.2
var_test_split = 0.1
var_batch_size = 8 

var_mtcnn_image_size = 160
var_mtcnn_margin = 0 

if not os.path.exists(var_dataset_split_folder ):
    os.makedirs(var_dataset_split_folder )
if not os.path.exists(var_dataset_split_folder  + '/train'):
    os.makedirs(var_dataset_split_folder +'/train')   
if not os.path.exists(var_dataset_split_folder  + '/validation'):
    os.makedirs(var_dataset_split_folder +'/validation') 
if not os.path.exists(var_dataset_split_folder  + '/test'):
    os.makedirs(var_dataset_split_folder +'/test')  

# 1. Dataset 

The following steps prepare the dataset by first calculating the mean and standard deviation and then generate triplets files. 

In [ ]:
run = f"python utils/calculate_rgb_mean_std.py\
                    --dir {var_dataset_folder}"
!{run}

In [ ]:
# mean and Standard dev 
mean = [0.6158, 0.4637, 0.3757]
std = [0.2124, 0.1863, 0.1812] 

In [ ]:
# Split persons in Train, Val and Test set by copying training data 
import shutil   

names = os.listdir(var_dataset_folder)   
n_samples = len(names) 
 
shuffled_indices = np.random.permutation(n_samples)
testset_inds = shuffled_indices[:int(n_samples * var_test_split)]
validationset_inds = shuffled_indices[int(n_samples * var_test_split) : int(n_samples * var_test_split) + int(n_samples * var_val_split)]
trainingset_inds = shuffled_indices[int(n_samples * var_test_split) + int(n_samples * var_val_split) :] 

print(len(testset_inds))
print(len(validationset_inds))
print(len(trainingset_inds))
print(names[testset_inds[0]])

for i in testset_inds: 
    shutil.copytree(f'{var_dataset_folder}/{names[i]}', f'{var_dataset_split_folder}/test/{names[i]}') 
for i in trainingset_inds: 
    shutil.copytree(f'{var_dataset_folder}/{names[i]}', f'{var_dataset_split_folder }/train/{names[i]}') 
for i in validationset_inds: 
    shutil.copytree(f'{var_dataset_folder}/{names[i]}', f'{var_dataset_split_folder }/validation/{names[i]}') 


In [ ]:
import random

class TripletGenerator: 
    def __init__(self, input_dir, output_file):  
        if not os.path.exists(input_dir): 
            raise Exception('Input dir does not exist!') 
        
        self.input_dir = input_dir 
            
        self.output_file = output_file
    
    def generate_triplets(self):    
        names = os.listdir(self.input_dir) 
    
        for name in names:  
            for file in os.listdir(self.input_dir + "/" + name):  
                in_filepath = '{}/{}/{}'.format(self.input_dir, name, file) 
                
                # TODO: Positive 
                positivearray = os.listdir(self.input_dir + "/" + name) 
                if len(positivearray) == 1: 
                    positive_file = positivearray[0] 
                else: 
                    idx_file = positivearray.index(file) 
                    del positivearray[idx_file] 
                    positive_file = random.choice(positivearray)
                
                #Negative    
                # First remove own name from list 
                namescopy = names.copy(); 
                idx = namescopy.index(name)  
                del namescopy[idx]
                
                random_name = random.choice(namescopy) 
                random_file = random.choice(os.listdir('{}/{}'.format(self.input_dir, random_name))) 
                
                print('anchor: {}, positive: {}, negative: {}'.format(file, positive_file, random_file)) 
                
                with open(self.output_file, 'a') as f:
                    f.write('{}\t{}\t{}\n'.format(file, positive_file, random_file))

        

In [ ]:
tgen_train = TripletGenerator(f'{var_dataset_split_folder}/train', 'triplets-train.txt') 
tgen_train.generate_triplets() 
tgen_test = TripletGenerator(f'{var_dataset_split_folder}/test', 'triplets-test.txt') 
tgen_test.generate_triplets() 
tgen_val = TripletGenerator(f'{var_dataset_split_folder}/validation', 'triplets-validation.txt') 
tgen_val.generate_triplets() 

In [ ]:
from torch.utils.data import Dataset  
import os 

class LFWDataset(Dataset): 
    
    def __init__(self, triplet_file, path): 
        self.triplets_file = triplet_file
        self.read_triplets()  
        self.path = path
        
    def read_triplets(self): 
        triplet_rows = open(self.triplets_file).read().splitlines()  
        self.triplets = [] 
        for triplet in triplet_rows: 
            split = triplet.split('\t')
            self.triplets.append({"anchor": split[0], "positive": split[1], "negative": split[2]})
        
    def __len__(self): 
        return len(self.triplets) 
    
    def person_from_path(self, path): 
        path = path.replace('augmented_', '').replace('bright_', '').replace('dark_', '').replace('hflip_', '').replace('mask_', '')
        type = path.split('.') 
        comp = type[0].split('_')
        
        o = "_" 
        return o.join(comp[:-1]) 
    
    def preprocess_image(self, path): 
        path = '{}{}/{}'.format(self.path, self.person_from_path(path), path) 
        ex_img = Image.open(path)  
        
        transform = transforms.Compose(
            [transforms.ToTensor(),
             transforms.Normalize(mean, std)]
        )
        image = transform(ex_img) 
        return image.cuda()
        
    def __getitem__(self, idx):  
        return self.preprocess_image(self.triplets[idx]['anchor']), self.preprocess_image(self.triplets[idx]['positive']), self.preprocess_image(self.triplets[idx]['negative'])

In [ ]:
lfwdataset_train = LFWDataset('triplets-train.txt', f'{var_dataset_split_folder}/train/') 
lfwdataset_test = LFWDataset('triplets-test.txt', f'{var_dataset_split_folder}/test/')  
lfwdataset_val = LFWDataset('triplets-validation.txt', f'{var_dataset_split_folder}/validation/') 

print(f'Train: {len(lfwdataset_train)}') 
print(f'Val: {len(lfwdataset_val)}') 
print(f'Test: {len(lfwdataset_test)}') 

In [ ]:
train_loader = torch.utils.data.DataLoader(dataset=lfwdataset_train,
                                           batch_size=var_batch_size,
                                           num_workers=0,
                                           shuffle=True, sampler=None,
                                           collate_fn=None)

test_loader = torch.utils.data.DataLoader(dataset=lfwdataset_test,
                                           batch_size=var_batch_size,
                                           num_workers=0,
                                           shuffle=False, sampler=None,
                                           collate_fn=None)

val_loader = torch.utils.data.DataLoader(dataset=lfwdataset_val,
                                           batch_size=var_batch_size,
                                           num_workers=0,
                                           shuffle=False, sampler=None,
                                           collate_fn=None)

# 2. Model  
Easily setup your model by choosing the right import 

In [ ]:
import torchvision.models as models   
import torch.nn as nn
#from facenet_pytorch import InceptionResnetV1
#from torchvision.models.inception_resnet_v1 import InceptionResnetV1
    
model = models.resnet34(pretrained=False)
model.fc = nn.Linear(512, 512) 
model = model.cuda() 

#model = models.resnet50(pretrained=False)   
#model = models.resnet18(pretrained=False) 

#from inception_resnet_v2 import Inception_ResNetv2
#model = Inception_ResNetv2()
#model = model.cuda() 

#model = InceptionResnetV1(pretrained='vggface2').eval().cuda()

# 3. Training

In [ ]:
loss_funct = nn.TripletMarginLoss(margin=1.0)

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001) 

In [ ]:
from datetime import datetime  
import os

class TensorBoardLogger: 
    
    def __init__(self, runname, date=datetime.today().strftime('%Y-%m-%d')):   
        path = 'logs/{}/{}'.format(date, runname)
        if not os.path.exists(path):
            os.makedirs(path) 
        self.writer = SummaryWriter(path) 
    
    def log_loss(self, mode, loss, epoch): 
        self.writer.add_scalar(f"loss/{mode}", loss, epoch)  
        self.writer.close()
    
    def log_acc(self, mode, acc, epoch): 
        self.writer.add_scalar(f"acc/{mode}", acc, epoch) 
        self.writer.close()
        
    def log_threshold_for_acc(self, mode, threshold, epoch): 
        self.writer.add_scalar(f"threshold/{mode}", threshold, epoch) 
        self.writer.close()

In [ ]:
class EmbeddingSpace:  
    
    """  
    :param model: The model that creates the image embeddings 
    :param tag: is the name the space will be stored in tensorboard 
    :param input_dir: directory where the images are stored that shall be loaded 
    :param number: Number of persons to consider - 10 persons with 10 images = 100 dots in the space  
    :param tensorboardlogger: The tensorboard writer created for this run 
    """
    def __init__(self, model, tag, input_dir, number, tensorboardlogger): 
        self.model = model 
        self.tag = tag 
        self.input_dir = input_dir 
        self.number = number  
        self.tensorboardlogger = tensorboardlogger
    
    """ 
    :param path: Image that is used to create a new embedding  
    """
    def calc_embedding(self, path):  
        image = Image.open(path) 
        transformed_image = transforms.ToTensor()(image).cuda() 
        return self.model.forward(transformed_image.unsqueeze(dim=0))   
    
    """ 
    Create the embedding space and add to tensorboard 
    """
    def create_space(self): 
        embeddings = [] 
        labels = []  
        
        persons = os.listdir(self.input_dir)[:self.number]  
        for name in persons:   
            for file in os.listdir("{}/{}".format(self.input_dir, name)):  
                in_filepath = '{}/{}/{}'.format(self.input_dir, name, file)  
                embeddings.append(self.calc_embedding(in_filepath)) 
                labels.append(name) 
        
        # Concat all embeddings into one tensor along axis 0 
        embedding_obj = torch.cat(embeddings, 0)
        
        self.tensorboardlogger.writer.add_embedding(embedding_obj, tag=self.tag, metadata=labels) 
        self.tensorboardlogger.writer.close()

In [ ]:
""" 
Performance measurements
"""  

class PerformanceMeasurement: 
    
    def __init__(self): 
        self.predictions = {}  
        self.thresholds = np.arange(0, 40, 0.1)   
    
    """ 
    Prediction inspired by faceguard pipeline 
    We calculate the distance between the anchor embedding and a 
    comparison embedding

    If the distance is larger than a threshold, we return that the 
    images are not the same (0) otherwise they are the same (1) 
    
    :param anchor_embedding: Embedding vector of the anchor 
    :param compare_embedding: Embedding vector to compare with the anchor 
    :param threshold: Highest distance until anchor and compare are the same  
    """
    def prediction(self, anchor_embedding, compare_embedding, threshold):  
        dist = np.linalg.norm(np.subtract(anchor_embedding.cpu().detach().numpy(), compare_embedding.cpu().detach().numpy()))   
        if dist > threshold: 
            return 0 
        else: 
            return 1 
    
    """ 
    Add new training run 
    :param anchor_embedding: Embedding vector of the anchor 
    :param anchor_embedding: Embedding vector of the same person as anchor  
    :param negative_embedding: Negative example 
    """
    def add_new_run(self, anchor_embedding, positive_embedding, negative_embedding):  
        for threshold in self.thresholds: 
            #Create entry if it does not exist
            if threshold not in self.predictions: 
                self.predictions[threshold] = {}   
                self.predictions[threshold]['tp'] = 0 
                self.predictions[threshold]['fp'] = 0  
                self.predictions[threshold]['tn'] = 0  
                self.predictions[threshold]['fn'] = 0 

            # The positive prediction (so comparison between anchor and positive) should return 1
            pred_pos = self.prediction(anchor_embedding, positive_embedding, threshold)  
            
            # The negative prediction (so comparison between anchor and negative) should return 0
            pred_neg = self.prediction(anchor_embedding, negative_embedding, threshold) 

            if pred_neg == 0: 
                self.predictions[threshold]['tn'] += 1  
            else: 
                self.predictions[threshold]['fp'] += 1 

            if pred_pos == 1: 
                self.predictions[threshold]['tp'] += 1 
            else: 
                self.predictions[threshold]['fn'] += 1  
    
    """ 
    Simple accuracy helper function 
    """
    def calc_acc(self, tp, tn, fp, fn):  
        return (tp+tn)/(tp+tn+fp+fn) 
    
    
    """ 
    Function that gets called after all runs 
    Here we calculate the accuracy for all thresholds and return the highest accuracy 
    
    This also helps us set the threshold for the pipeline function 
    """
    def calc_total_acc(self):  
        max_acc = 0  
        max_t = 0  
        
        for threshold in self.predictions: 
            acc = self.calc_acc(self.predictions[threshold]['tp'], self.predictions[threshold]['tn'], self.predictions[threshold]['fp'], self.predictions[threshold]['fn']) 
            if acc > max_acc:  
                print(f"Trehshold {threshold} > Acc: {acc}") 
                max_acc = acc 
                max_t = threshold
        
        return max_acc, max_t
    

In [ ]:
from tqdm import tqdm  

class Trainer: 
    def __init__(self, run_tag, model, train_loader, valid_loader, mean, std, optimizer, loss_funct, measure_performance_of_epochs, create_embedding_space=False): 
        
        self.model = model
        self.train_loader = train_loader
        self.valid_loader = valid_loader  
        self.tag = run_tag
         
        self.mean = mean 
        self.std = std 
        
        self.logger = TensorBoardLogger(run_tag) 

        self.optimizer = optimizer
        self.loss_func = loss_funct
        
        self.train_loss = [] 
        self.validation_loss = []
        self.acc = [] 
        
        self.measure_performance = measure_performance_of_epochs
        
        # if the optimizer is not initialzed 
        if optimizer:
            self.optimizer = optimizer
        else:
            self.optimizer = torch.optim.Adam(**optimizer_args)

        
    def train_epoch(self): 
        total_loss = 0  
        
        perf_measure = PerformanceMeasurement() 
        
        for i, data in enumerate(self.train_loader):  
            anchor, positive, negative = data  
        
            anchor_embedding = self.model.forward(anchor)  
            positive_embedding = self.model.forward(positive) 
            negative_embedding = self.model.forward(negative)  
            
            self.optimizer.zero_grad() 
             
            loss = loss_funct(anchor_embedding, positive_embedding, negative_embedding)
            total_loss += loss.item() 
            loss.backward()            
            
            if self.measure_performance: 
                for embedding in range(len(anchor_embedding)): 
                    perf_measure.add_new_run(anchor_embedding[embedding], positive_embedding[embedding], negative_embedding[embedding]) 
        

            self.optimizer.step() 
            if i % 40 == 0: 
                print("Batchno {} - {}%".format(i, ((i / len(self.train_loader))*100))) 
        
        return (total_loss) / len(self.train_loader), perf_measure.calc_total_acc() 
    
    def train(self, n_epochs, start_epoch=0): 
        print(f"Starting to train {n_epochs} epochs")
        for e in range(n_epochs): 
            print("Starting epoch {} with {} batches".format(e+start_epoch, len(self.train_loader)))
            
            loss, trainacc = self.train_epoch()  
            self.logger.log_loss('train', loss, e+start_epoch)  
            self.logger.log_acc('train', trainacc[0], e+start_epoch) 
            self.logger.log_threshold_for_acc('train', trainacc[1], e+start_epoch)
            
            self.train_loss.append(loss) 
            
            vallos, valacc = self.evaluate()   
            self.validation_loss.append(vallos) 
            self.logger.log_loss('validation', vallos, e+start_epoch)  
            self.logger.log_acc('validation', valacc[0], e+start_epoch) 
            self.logger.log_threshold_for_acc('validation', valacc[1], e+start_epoch)
            
            print(f"Epoch {e+start_epoch} completetd with trainloss {loss}, trainacc {trainacc} and validationloss: {vallos}, validationacc {valacc}") 
            self.plotloss() 
            
            if create_embedding_space == True: 
                embeddingspace = EmbeddingSpace(self.model, f"{self.tag}-epoch-{e}", 'dataset/lfw/lfw_cropeed', 10, self.logger) 
                embeddingspace.create_space()  
                print("Created embedding space for epoch.")
    
    def evaluate(self):   
        print("Starting evalutaion")
        eval_total_loss = 0  
        perf_measure = PerformanceMeasurement() 
        
        with torch.no_grad(): 
            for i, data in enumerate(self.train_loader):  
                anchor, positive, negative = data  

                #print("{} - Training {} ".format(i, anchor))
                anchor_embedding = self.model.forward(anchor)  
                positive_embedding = self.model.forward(positive) 
                negative_embedding = self.model.forward(negative)  
                
                if self.measure_performance: 
                    for embedding in range(len(anchor_embedding)): 
                        perf_measure.add_new_run(anchor_embedding[embedding], positive_embedding[embedding], negative_embedding[embedding]) 
                
                loss = loss_funct(anchor_embedding, positive_embedding, negative_embedding)
                eval_total_loss += loss.item()   
                
        return (eval_total_loss) / len(self.valid_loader), perf_measure.calc_total_acc() 
    
    def plotloss(self): 
        fig, axs = plt.subplots(2)  
        axs[0].plot(np.array(self.train_loss)) 
        axs[0].set_title('Training loss')  
        axs[1].plot(np.array(self.validation_loss)) 
        axs[1].set_title('Validation loss')  
        fig.tight_layout(pad=2.0)
        plt.show() 
    

In [ ]:
trainer = Trainer('resnet-34-tripletloss-run-1-batchsize-8', model, train_loader, val_loader, mean, std, optimizer, loss_funct, False, False) 
trainer.train(30, 0)

In [ ]:
torch.save(model.state_dict(), 'inception-resnet-v2-no-augmentation-tripletloss-new-split-run-1-wmeanandstd-batchsize-8.pth')

# 4. Evaluation

In [ ]:
def run_test():  
    perf_measure_test = PerformanceMeasurement()   
    print("Starting test")
    for i, data in enumerate(test_loader):   
        anchor, positive, negative = data  

        anchor_embedding = model.forward(anchor)  
        positive_embedding = model.forward(positive) 
        negative_embedding = model.forward(negative)  
                
        for embedding in range(len(anchor_embedding)): 
            perf_measure_test.add_new_run(anchor_embedding[embedding], positive_embedding[embedding], negative_embedding[embedding]) 
    
    if i % 40 == 0: 
        print("Batchno {} - {}%".format(i, ((i / len(self.test_loader))*100)))  
    
    return perf_measure_test.calc_total_acc() 

logger = TensorBoardLogger('inception-resnet-v1-pretrained') 
acc = run_test()  
print(acc)
logger.log_acc('test', acc[0], 0) 
logger.log_threshold_for_acc('test', acc[1], 0)